## Model with multiple Depth

In [6]:
from model_zoo.models import AutoEncoder
from utils.utils import load_config
import torch

## Open a model 

In [15]:
path_dir = "/home/ubuntu/project/sentinel-2-ai-compressor/src/results/2025-08-10_09-30-54"

weights_path = f"{path_dir}/checkpoints/best_model.pth"
config = load_config(config_path=f"{path_dir}/config.yaml")

model = AutoEncoder(
            in_channels=len(config['DATASET']['bands']),
            out_channels=len(config['DATASET']['bands']),
            base_channels=config['MODEL']['base_channels'],
            depth=config['MODEL']['depth'],
            bottleneck_factor=config['MODEL']['bottleneck_factor'],
        )

In [16]:
# Count parameters
total_params, trainable_params = model.count_parameters()
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: {total_params * 4 / (1024**2):.1f} MB (float32)")

# Test forward pass
model.eval()
with torch.no_grad():
    # Radar input tensor
    x = torch.randn(1, 3, 1024, 1024)
    output  = model(x)

    # Compression analysis
    bottleneck = model.get_bottleneck_tensor()
    compression_info = model.get_compression_info(x, bottleneck)

    print(f"\nArchitecture:")
    print(f"Channel progression: {model.channel_progression}")
    print(f"Bottleneck channels: {model.bottleneck_channels}")

    print(f"Bottleneck: {bottleneck.shape}")
    print(f"Output: {output.shape}")

    print(f"\nCompression Analysis:")
    print(f"Input: {compression_info['input_elements']:,} elements")
    print(f"Bottleneck: {compression_info['bottleneck_elements']:,} elements")
    print(f"Compression ratio: {compression_info['compression_ratio']:.2f}x")
    print(f"Size reduction: {compression_info['reduction_percent']:.1f}%")

    # Memory usage
    input_mb = compression_info['input_elements'] * 4 / (1024**2)
    bottleneck_mb = compression_info['bottleneck_elements'] * 4 / (1024**2)
    print(f"\nMemory usage:")
    print(f"Input: {input_mb:.1f} MB")
    print(f"Bottleneck: {bottleneck_mb:.1f} MB")

Total parameters: 118,813,871
Trainable parameters: 118,813,871
Model size: 453.2 MB (float32)

Architecture:
Channel progression: [3, 64, 128, 256, 512, 1024, 1024, 1024]
Bottleneck channels: 512
Bottleneck: torch.Size([1, 1024, 8, 8])
Output: torch.Size([1, 3, 1024, 1024])

Compression Analysis:
Input: 3,145,728 elements
Bottleneck: 65,536 elements
Compression ratio: 48.00x
Size reduction: 97.9%

Memory usage:
Input: 12.0 MB
Bottleneck: 0.2 MB
